# Preliminaries Install/Import

In [2]:
import json
# import re
# import numpy as np 
import pandas as pd 



# import os
import ujson
import msgpack
# import logging
# import math
import multiprocessing
import sys
sys.path.append('/kaggle/input/tabert-datagen/')
# from modeling_t5dec import *
# from table import *
import time
from pathlib import Path
from typing import Dict, Optional, Iterator, Set, Tuple
# try:
#     import redis
# except:
#     !pip install redis
#     import redis
import wandb
import torch
import zmq
# from table_bert.utils import BertTokenizer
# from table_bert.config import TableBertConfig
from torch.utils.data import Dataset
# from torch.utils.data.sampler import Sampler
# import torch.distributed as dist
from tqdm import tqdm
# from table_bert.table import Column, Table
# import h5py

from datasets import Dataset
import transformers
from transformers import AutoTokenizer, EncoderDecoderModel, Seq2SeqTrainingArguments, Seq2SeqTrainer, PreTrainedTokenizerBase, GenerationConfig
import safetensors

import torch
from torch import nn
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [3]:
# from kaggle_secrets import UserSecretsClient

# user_secrets = UserSecretsClient()

# my_secret = user_secrets.get_secret("wandb_api_key") 

wandb.login(key='секрет')

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

# Data extraction

In [4]:
INPUT_FILE = '/kaggle/input/ruwiki-tables-and-lists/wiki_tables_and_lists.jsonl'
df = pd.DataFrame(columns=['text', 'summary'])
NUM_SAMPLES = 10
with open(INPUT_FILE, "r", encoding="utf-8") as f:
        # Пропускаем уже обработанные таблицы
        # for _ in range(processed_tables_count):
        #     f.readline()
        n = 0
        for line in tqdm(f):
            line = line.strip()
            if not line:
                continue
            entry = json.loads(line)
            uuid = entry.get("uuid", "")
            context_before = entry.get("context_before", "") or ""
            caption = entry.get("caption", "") or ""
            header = entry.get("header", [])
            data = entry.get("data", [])

            header_names = [col["name"] for col in header]
            header_line = " | ".join(header_names)
            context_text = '\n'.join('\n'.join(inner_text) for inner_text in context_before)
            table_text = "\n".join([" | ".join([unit[1] for unit in row]) for row in data])
            # full_text = f"{context_text}\n{caption}\n{header_line}\n{table_text}".strip()
            full_text = f"{caption}\n{header_line}\n{table_text}".strip()
            df.loc[len(df)] = [table_text, context_text]
            n += 1
            if n == NUM_SAMPLES:
                break
f.close()
df.sample(5)


9it [00:00, 529.54it/s]


,text,summary
6,"Соционика — типология коллективов, групп людеи...",Предмет изучения\nОпределения предмета социони...
1,Вильнюсский уезд\nКаунасский уезд\nКлайпедс...,Территория Литвы разделена на 60 самоуправлени...
4,основного общего образования (средняя школа) (...,Первоначально греческое σχολή означало «досуг...
3,Каунас (298 800);\nКлайпеда (152 000);\nШяуля...,Города Литвы\nВ Литве выделяются 4 типа населе...
5,Логотипом кофе «Гиндукуш» является слон.\nИнди...,Образ слона в культуре\nВ русском языке слон а...


In [5]:
data = Dataset.from_pandas(df, preserve_index=False)
data.to_csv('data.csv')

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

10944

In [6]:
%%time
tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/rubert-base-cased")
# tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-uncased")
tokenizer.bos_token = tokenizer.cls_token
tokenizer.eos_token = tokenizer.sep_token
# db = TableDatabase.from_jsonl(file_path= '/kaggle/input/dataset-demo/wiki_dump.jsonl',
#                               tokenizer= tokenizer,
#                              backend='memory',
#                              num_entries= 100,
#                              num_workers= 7)

tokenizer_config.json:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

CPU times: user 273 ms, sys: 71.2 ms, total: 344 ms
Wall time: 920 ms


In [7]:
%%time
def preprocess_function(examples):
    model_inputs = tokenizer(examples["text"], max_length=512, padding='max_length', truncation=True, return_tensors='pt')
    
    labels = tokenizer(text_target=examples["summary"], max_length=512, padding='max_length', truncation=True, return_tensors='pt')
    model_inputs["labels"] = labels["input_ids"]
    print(model_inputs["labels"])
    # model_inputs["labels"] = model_inputs["labels"][1:]
    return model_inputs
tokenized_data = data.map(preprocess_function, batched=True).remove_columns(['text', 'summary'])
tokenized_data.set_format(type="torch")


Map:   0%|          | 0/10 [00:00<?, ? examples/s]

tensor([[   101,    843,    122,  ...,      0,      0,      0],
        [   101,  24758,  21662,  ...,      0,      0,      0],
        [   101,  52063,  33539,  ...,      0,      0,      0],
        ...,
        [   101, 118939,  53559,  ...,      0,      0,      0],
        [   101,  80512,  20420,  ...,      0,      0,      0],
        [   101,  23878,  16767,  ...,      0,      0,      0]])
CPU times: user 72.2 ms, sys: 16.2 ms, total: 88.5 ms
Wall time: 130 ms


In [8]:
df = pd.DataFrame(columns=['text', 'summary'])
for example in iter(db):
    table = example.get_table()
    context = ' '.join([' '.join(row) for row in example.context[0]])
    table = ' '.join([' '.join(row) for row in table.as_row_list])
    df.loc[len(df)] = [table, context]



In [9]:
tokenized_data = tokenized_data.train_test_split(0.2)

In [10]:
test_dataset = tokenized_data['test']
train_dataset, val_dataset = tokenized_data['train'].train_test_split(0.1).values()

# Model definition

In [11]:
class ChillModel(EncoderDecoderModel):
    def __init__(self,
                 config: Optional[transformers.configuration_utils.PretrainedConfig] = None,
                 encoder: Optional[transformers.modeling_utils.PreTrainedModel] = None,
                 decoder: Optional[transformers.modeling_utils.PreTrainedModel] = None):
        super(ChillModel, self).__init__(config, encoder, decoder)

    @torch.no_grad()
    def top_k_tables(query: str,
                     faiss_db,
                     k: int,
                     tokenizer: PreTrainedTokenizerBase,
                     generation_config: GenerationConfig):
        input = tokenizer(query)
        query_emb = self.encoder(**input)

        #-- find k nearest tables
        # tables, tables_emb = 
        self.decoder.generate(encoder_output= tables_emb, generation_config= generation_config)
        
        return k_nearest_tables, tables_sum

    

In [12]:
import gc
gc.collect()
torch.cuda.empty_cache()
gc.collect()

In [13]:
model = ChillModel.from_encoder_decoder_pretrained('DeepPavlov/rubert-base-cased', 'DeepPavlov/rubert-base-cased')
model.config.decoder_start_token_id = tokenizer.bos_token_id
model.config.eos_token_id = tokenizer.eos_token_id
model.config.pad_token_id = tokenizer.pad_token_id

if torch.cuda.device_count() > 1:
    print(f"Using {torch.cuda.device_count()} GPUs for training...")
    model = nn.DataParallel(model)
model = model.to(device)


# Training

In [14]:
# 3449dfce11d8fe7fc2b677ff852a8d36414bef0b
BATCH_SIZE = 32
NUM_EPOCHS = 1000
NAME = 'bert2bert' +f"_{BATCH_SIZE}b" + f'_{NUM_EPOCHS}e'
SAVE_DIR = './results'

training_args = Seq2SeqTrainingArguments(
    output_dir= SAVE_DIR,
    eval_strategy= 'epoch',
    prediction_loss_only= True,
    learning_rate= 2e-5,
    per_device_train_batch_size= BATCH_SIZE,
    per_device_eval_batch_size= BATCH_SIZE,
    num_train_epochs= NUM_EPOCHS,
    weight_decay= 0.01,
    remove_unused_columns= True,
    logging_strategy= 'epoch',
    save_strategy= 'epoch', 
    load_best_model_at_end= True,
    run_name= NAME,
    predict_with_generate= False,
    save_total_limit= 2)
    # auto_find_batch_size= True

trainer = Seq2SeqTrainer(model,
                         processing_class= tokenizer,
                         args= training_args,
                         train_dataset= train_dataset,
                         eval_dataset = val_dataset,
                        )
trainer.train()
trainer.state.best_model_checkpoint

In [15]:
model.config.begin_suppress_tokens = [model.config.decoder_start_token_id]

In [16]:
print(model.config.decoder_start_token_id)
print(model.config.eos_token_id)
model.config.pad_token_id

In [17]:
tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/rubert-base-cased")
tokenizer.bos_token = tokenizer.cls_token
tokenizer.eos_token = tokenizer.sep_token

In [26]:
generation_config = transformers.GenerationConfig(
    max_length=256,
    min_length=None,
    do_sample=False,
    early_stopping=True,
    num_beams=1,
    temperature=1,
    top_k=None,
    top_p=None,
    length_penalty=1.0,  # > 1.0 longer sequences, < 1.0 shorter sequences
    num_return_sequences=1,
    max_time=None,  # in seconds
    num_beam_groups=1,
    output_scores=False,
    bos_token_id= model.config.decoder_start_token_id,
    begin_suppress_tokens= [model.config.decoder_start_token_id])

/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:638: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


In [ ]:
generation_config

In [ ]:
inputs = tokenizer('Михаил Ломоносов основал МГУ', return_tensors="pt").to(device)
input_ids = inputs.input_ids
with torch.no_grad():
    emb = model.encoder(**inputs)
emb.keys()

In [25]:
model = EncoderDecoderModel.from_pretrained('/kaggle/input/bert2bert-4b-300e/results/checkpoint-9912').to(device)

In [27]:
df = pd.read_csv('/kaggle/input/scores-output/output.csv')
df.sample(3)

,Prompt,Context,Result
36,Список наиболее значимых битв Средневековья,ВикипедияСтатьи годаИтоги\nНоминация «Статьи.\...,Из предоставленного контекста невозможно соста...
14,Список действующих вулканов Камчатки,Вулканы КабоВерде\nСписок действующих и потух...,"Извините, но у меня нет информации о вулканах ..."
11,"Списки рек, протекающих через Москву",год\n* В России летом произошли природные ката...,"К сожалению, в предоставленном контексте нет и..."


In [28]:
results = df['Result'].values

In [29]:
questions = df['Prompt'].values

In [45]:
# from transformers import EncoderDecoderModel, BertTokenizer
# import torch



# training

# model.config.pad_token_id = tokenizer.pad_token_id
# model.config.vocab_size = model.config.decoder.vocab_size

# input_ids = tokenizer("This is a really long text", return_tensors="pt").input_ids
# labels = tokenizer("This is the corresponding summary", return_tensors="pt").input_ids
# outputs = model(input_ids=input_ids, labels=labels)
# loss, logits = outputs.loss, outputs.logits

# save and load from pretrained
# model.save_pretrained("bert2bert")
# model = EncoderDecoderModel.from_pretrained("bert2bert")
outputs = []

for quesion in tqdm(questions):
    inputs = tokenizer(quesion, return_tensors="pt").to(device)
    with torch.no_grad():
        emb = model.encoder(**inputs).last_hidden_state
        
    # print(input_ids)
    
    # input_ids = next(iter(train_dataset))['input_ids']
    # generation
    generated = model.decoder.generate(encoder_hidden_states = emb, generation_config=generation_config)
    outputs.append(tokenizer.batch_decode(generated,  skip_special_tokens= True))

  0%|          | 0/41 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:638: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
100%|██████████| 41/41 [02:33<00:00,  3.75s/it]


In [46]:
generated = tokenizer.batch_decode(generated,  skip_special_tokens= True)

In [47]:
try:
    from evaluate import load
except:
    !pip install evaluate
    from evaluate import load

In [34]:
!pip install bert_score

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.3 MB/s eta 0:00:00


In [64]:
questions[0]

'Численность населения Москвы за последние 20 лет'

In [71]:
outputs

[['См. такжеПримечанияСсыл # # ки2014года2017года.2012года.2017года.2017года.2017года.2013годапо2013года —2017года.2013года —..2017года —..2017года —2017года..2017года —2017года.2017года —2017года.2017года —2017года.2017года —2017года.2017года —2017года —2017года2017года —2017года.2017года —2017года —2017года —2017года2017года —2017года2013года —2017'],
 ['Списокгубернских Федерациивклассификации видовспортивныхобъектов, входящихв список. Списокнаселенныхобъектоввразличныхобъектахвразличныхобъектахвразличныхобъектах. СписокнаселенныхобъектоввклассификациинаселенныхобъектоввРоссии.. # # n. # # # # n # # # # # # # # # # # # # # # # # #в # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #'],
 ['НаградыПримечанияСсыл # # ки # # s # # s, # # os # # s1907года # # # # os # # s1907годаподназванием " # # # # # # # # " 彬 # # # # # # # # " # # # # # # # # # #.....201

In [75]:

bertscore = load("bertscore")
predictions, references = [o[0] for o in outputs], questions

bs = bertscore.compute(predictions=predictions, references=references, lang="ru")

In [80]:
print('precision', sum(bs['precision'])/len(bs['precision']))
print('recall', sum(bs['recall'])/len(bs['recall']))
print('f1', sum(bs['f1'])/len(bs['f1']))

precision 0.41671923166368063
recall 0.5871573090553284
f1 0.48525058496289136


In [74]:
from ignite.metrics import Rouge
m = Rouge(variants=["L"], multiref="average")


m.update((predictions, references))

print(m.compute())

{'Rouge-L-P': 0.0031532775377011084, 'Rouge-L-R': 0.7919307892145258, 'Rouge-L-F': 0.7919307892145258}


In [ ]:
# tokenizer.batch_decode(generated, skip_special_tokens= True)

In [ ]:
 # [tokenizer.decode(t, skip_special_tokens=True) for t in generated]

In [ ]:
# ''.join(tokenizer.convert_ids_to_tokens(generated[0]))